In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
os.environ['TF_NUM_INTRAOP_THREADS'] = '0'
os.environ['TF_NUM_INTEROP_THREADS'] = '0'
os.environ['TF_KERAS'] = '1'
os.environ['TF_MKL'] = '1'

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


In [2]:
# Load the dataset
data = pd.read_csv('review.csv')

In [3]:
data.head()

,sentiment,review
0,1.0,Good but need updates and improvements
1,0.0,"Worst mobile i have bought ever, Battery is dr..."
2,1.0,when I will get my 10% cash back.... its alrea...
3,1.0,Good
4,0.0,The worst phone everThey have changed the last...


In [4]:
data.tail()

,sentiment,review
14670,1.0,"I really like the phone, Everything is working..."
14671,1.0,The Lenovo K8 Note is awesome. It takes best p...
14672,1.0,Awesome Gaget.. @ this price
14673,1.0,This phone is nice processing will be successf...
14674,1.0,Good product but the pakeging was not enough.


In [5]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], test_size=0.001)

In [6]:
# Tokenize the text and pad the sequences
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=100, padding='post', truncating='post')
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=100, padding='post', truncating='post')

In [7]:
# Define the model
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    tf.keras.layers.GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

In [8]:
# Compile the model
opt = tf.compat.v1.train.AdamOptimizer()
opt = tf.compat.v1.train.experimental.enable_mixed_precision_graph_rewrite(opt)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [9]:
# Train the model
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), use_multiprocessing=True)

Epoch 1/5
459/459 [==============================] - 11s 15ms/step - loss: 0.5115 - accuracy: 0.7542 - val_loss: 0.3412 - val_accuracy: 0.8000
Epoch 2/5
459/459 [==============================] - 7s 15ms/step - loss: 0.3231 - accuracy: 0.8739 - val_loss: 0.3229 - val_accuracy: 0.9333
Epoch 3/5
459/459 [==============================] - 6s 13ms/step - loss: 0.2818 - accuracy: 0.8928 - val_loss: 0.2717 - val_accuracy: 0.8667
Epoch 4/5
459/459 [==============================] - 6s 14ms/step - loss: 0.2582 - accuracy: 0.9016 - val_loss: 0.2800 - val_accuracy: 0.9333
Epoch 5/5
459/459 [==============================] - 7s 15ms/step - loss: 0.2389 - accuracy: 0.9137 - val_loss: 0.2312 - val_accuracy: 0.9333


In [10]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 20ms/step - loss: 0.2312 - accuracy: 0.9333


In [11]:
# Print the model's accuracy
print('Test accuracy:', accuracy)

Test accuracy: 0.9333333373069763


In [15]:
# Get review from user input
user_review = input("Enter your review: ")

# Preprocess user review
user_review = tokenizer.texts_to_sequences([user_review])
user_review = pad_sequences(user_review, maxlen=100, padding='post', truncating='post')

Enter your review: phone is good but the camera is bad


In [16]:
# Predict sentiment for user review
sentiment = model.predict(user_review)

# Print the predicted sentiment
print(sentiment)
if sentiment > 0.6:
    print("Positive")
elif sentiment < 0.6 and sentiment >= 0.1:
    print("Neutral")
else:
    print("Negative")

1/1 [==============================] - 0s 9ms/step
[[0.3004099]]
Neutral
